In [21]:
from __future__ import print_function

import logging
import os

import SimpleITK as sitk
import six
import numpy as np

import radiomics
from radiomics import featureextractor, getFeatureClasses


In [22]:


def tqdmProgressbar():
  """
  This function will setup the progress bar exposed by the 'tqdm' package.
  Progress reporting is only used in PyRadiomics for the calculation of GLCM and GLSZM in full python mode, therefore
  enable GLCM and full-python mode to show the progress bar functionality

  N.B. This function will only work if the 'click' package is installed (not included in the PyRadiomics requirements)
  """
  global extractor

  radiomics.setVerbosity(logging.INFO)  # Verbosity must be at least INFO to enable progress bar

  import tqdm
  radiomics.progressReporter = tqdm.tqdm


def clickProgressbar():
  """
  This function will setup the progress bar exposed by the 'click' package.
  Progress reporting is only used in PyRadiomics for the calculation of GLCM and GLSZM in full python mode, therefore
  enable GLCM and full-python mode to show the progress bar functionality.

  Because the signature used to instantiate a click progress bar is different from what PyRadiomics expects, we need to
  write a simple wrapper class to enable use of a click progress bar. In this case we only need to change the 'desc'
  keyword argument to a 'label' keyword argument.

  N.B. This function will only work if the 'click' package is installed (not included in the PyRadiomics requirements)
  """
  global extractor

  # Enable the GLCM class to show the progress bar
  extractor.enableFeatureClassByName('glcm')

  radiomics.setVerbosity(logging.INFO)  # Verbosity must be at least INFO to enable progress bar

  import click

  class progressWrapper:
    def __init__(self, iterable, desc=''):
      # For a click progressbar, the description must be provided in the 'label' keyword argument.
      self.bar = click.progressbar(iterable, label=desc)

    def __iter__(self):
      return self.bar.__iter__()  # Redirect to the __iter__ function of the click progressbar

    def __enter__(self):
      return self.bar.__enter__()  # Redirect to the __enter__ function of the click progressbar

    def __exit__(self, exc_type, exc_value, tb):
      return self.bar.__exit__(exc_type, exc_value, tb)  # Redirect to the __exit__ function of the click progressbar

  radiomics.progressReporter = progressWrapper



In [18]:

# Get some test data
testCase = 'lung2'
# repositoryRoot points to the root of the repository. The following line gets that location if this script is run
# from it's default location in \pyradiomics\bin. Otherwise, it will point to some (invalid) folder, causing the
# getTestCase function to fail to find the test case in the repository. In that case, a test case will be downloaded to
# temporary files and it's location is returned.
repositoryRoot = os.path.abspath(os.path.join(os.getcwd(), ".."))
# imageName, maskName = radiomics.getTestCase(testCase, repositoryRoot)
imageName0 = 'nrrd/brain1_image.nrrd'
maskName0 = 'nrrd/brain1_label.nrrd'


In [19]:
imageName = sitk.ReadImage(imageName0)
maskName = sitk.ReadImage(maskName0)

In [28]:
size = 256
imageName = sitk.GetImageFromArray(np.random.randn(size, size, size))
maskV = np.ones((size, size, size))
maskV[0][0][0] = 0
maskName = sitk.GetImageFromArray(maskV)


In [30]:
sitk.GetArrayFromImage(maskName).min()

np.float64(0.0)

In [31]:

# Get the location of the example settings file
# paramsFile = os.path.abspath(r'exampleSettings\exampleVoxel.yaml')
paramsFile = os.path.abspath(r'exampleVoxel.yaml')

if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
  print('Error getting testcase!')
  exit()

# Regulate verbosity with radiomics.verbosity
# radiomics.setVerbosity(logging.INFO)

# Get the PyRadiomics logger (default log-level = INFO
logger = radiomics.logger
logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

# Write out all log entries to a file
handler = logging.FileHandler(filename='testLog.txt', mode='w')
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

# Initialize feature extractor using the settings file
extractor = featureextractor.RadiomicsFeatureExtractor(paramsFile)
featureClasses = getFeatureClasses()

# Uncomment one of these functions to show how PyRadiomics can use the 'tqdm' or 'click' package to report progress when
# running in full python mode. Assumes the respective package is installed (not included in the requirements)

tqdmProgressbar()
# clickProgressbar()

print("Active features:")
for cls, features in six.iteritems(extractor.enabledFeatures):
  if features is None or len(features) == 0:
    features = [f for f, deprecated in six.iteritems(featureClasses[cls].getFeatureNames()) if not deprecated]
  for f in features:
    print(f)
    print(getattr(featureClasses[cls], 'get%sFeatureValue' % f).__doc__)


Loading parameter file d:\Desktop\workspace\todo\exampleVoxel.yaml


Active features:
JointEntropy

    **12. Joint Entropy**

    .. math::
      \textit{joint entropy} = -\displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}
      {p(i,j)\log_2\big(p(i,j)+\epsilon\big)}


    Joint entropy is a measure of the randomness/variability in neighborhood intensity values.

    .. note::
      Defined by IBSI as Joint entropy
    


In [32]:
labels = np.unique(sitk.GetArrayFromImage(maskName))
len(labels)

2

In [33]:

print("Calculating features")
featureVector = extractor.execute(imageName, maskName, voxelBased=True)


Starting voxel based extraction
Calculating features with label: 1
Loading image and mask


Calculating features


Adding image type "Original" with custom settings: {}
Calculating features for original image
Computing glcm
batch: 100%|██████████| 1678/1678.0 [02:44<00:00, 10.19it/s]


In [11]:
featureVector

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'),
             ('diagnostics_Versions_Numpy', '2.0.0'),
             ('diagnostics_Versions_SimpleITK', '2.4.0'),
             ('diagnostics_Versions_PyWavelet', '1.7.0'),
             ('diagnostics_Versions_Python', '3.10.0'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': True,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True,
               'binWidth': 25,
               'kernelRadius': 2,
               'maskedKernel': True,
  

In [34]:
image = featureVector['original_glcm_JointEntropy']
image_array = sitk.GetArrayFromImage(image)

image_array.mean()

np.float64(nan)

In [ ]:

for featureName, featureValue in six.iteritems(featureVector):
  if isinstance(featureValue, sitk.Image):
    sitk.WriteImage(featureValue, '%s_%s.nrrd' % (testCase, featureName))
    print('Computed %s, stored as "%s_%s.nrrd"' % (featureName, testCase, featureName))
  else:
    print('%s: %s' % (featureName, featureValue))